In [ ]:
from keras.models import Sequential
import keras
from keras.utils import np_utils
import preprocessing
import postprocessing
import os
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import pickle


## Load the the preprocessed data

In [ ]:
zeroVec = np.zeros(72,int)
sequence_length = 100


In [ ]:
#load the data from saved files
network_input = np.loadtxt('processed_200_filtered_net_input_final.txt', dtype=int)
network_output = np.loadtxt('processed_200_filtered_net_output_final.txt', dtype=int)
with open('processed_200_filtered_vocabularies_final.pkl','rb') as f: 
    str_to_int, int_to_str, count_int = pickle.load(f) #configuration to category conversions and counts 

print(network_input.shape)

## Examine the data

In [ ]:
import matplotlib.pyplot as plt
dict_keys = [k for k,v in sorted([(k,v) for k,v in count_int.items()], key=lambda tup: tup[1])[::-1]]
dict_vals = sorted([v for k,v in count_int.items()])[::-1]
#show the 10 most common note configurations
plt.bar(dict_keys[:10], dict_vals[:10], color='g')
plt.show()

In [ ]:
# Shuffle data
n = network_input.shape[0]
I = np.random.permutation(n)

#define test and test sizes
ntr = 20000
nts = 5000

X = network_input[I[:ntr+nts]]
y = network_output[I[:ntr+nts]]

nclass = np.maximum(np.max(X), np.max(y)) #amount of different configurations in the data

#one-hot code
X = np_utils.to_categorical(X, num_classes=nclass+1) 
y = np_utils.to_categorical(y, num_classes=nclass+1)

#split to train and test
Xtr = X[:ntr]
ytr = y[:ntr]
Xts = X[ntr:ntr+nts]
yts = y[ntr:ntr+nts]

## train / load model

In [ ]:
import keras.backend as K
K.clear_session()

In [ ]:
final_model = Sequential()
final_model.add(keras.layers.LSTM(
        3,
        input_shape=(Xtr.shape[1], Xtr.shape[2]),
        return_sequences=True
    ))
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.LSTM(128,return_sequences=True)) #sub with catego
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.LSTM(128))
final_model.add(keras.layers.Dropout(0.3))
final_model.add(keras.layers.Dense(ytr.shape[1]))
final_model.add(keras.layers.Activation('softmax'))



pwd = os.path.dirname(os.path.realpath('__file__'))
filepath = os.path.join(pwd, "models", "weights-improvement-{epoch:02d}-{loss:.4f}-waitingModel-alldata.hdf5")   
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=1,        
    save_best_only=True,        
    mode='min')

callbacks_list = [checkpoint]     


# load the network weights

filename = "weights-improvement-160-1.1110-alldata_final.hdf5"
model_path = os.path.join(pwd, "models", filename)
#final_model.load_weights(model_path) #use this line to load the weights of the pretrained model

final_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
hist = final_model.fit(Xtr, ytr, epochs=400, batch_size=256, callbacks=callbacks_list, validation_data=(Xts,yts)) #use this line to train a new model

final_model.summary()

In [ ]:
import random
ind = random.randint(0,Xts.shape[0])
print("Testing test pattern at {0}".format(ind))
nt = 500
ynew = np.zeros(nt)
pat_len = X.shape[1]
pat = np.reshape(Xts[ind], (1,Xts.shape[1],Xts.shape[2]))
patC = np.copy(pat)
res = np.copy(pat)
print(res.shape)
for t in range(nt):
    prob = final_model.predict(pat)
    pred = np.argmax(prob)
    print(pred,end=",")
    newNote = np.zeros((1,1,X.shape[2]))
    newNote[0,0,pred] = 1.0
    pat = np.append(pat,newNote,axis=1)
    res = np.append(res,newNote,axis=1)
    pat = pat[:,1:sequence_length+1,:]

print()
print(res[0].shape)
noteMat = np.zeros((res[0].shape[0],72),dtype=int)
for veci, vec in enumerate(res[0]):
    nextNoteInt = 0
    for i,e in enumerate(vec):
        if e > 0:
            nextNoteInt = int(i)
            break
    noteMat[veci] = np.array([int(b) for b in int_to_str[nextNoteInt]])
noteMat[-1] = np.zeros(72,dtype=int)
postprocessing.convertMatToTrack(noteMat.T,32,"prediction.mid")


noteMat = np.zeros((100,72),dtype=int)
for veci, vec in enumerate(patC[0]):
    nextNoteInt = 0
    for i,e in enumerate(vec):
        if e > 0:
            nextNoteInt = int(i)
            break
    noteMat[veci] = np.array([int(b) for b in int_to_str[nextNoteInt]])
noteMat[-1] = np.zeros(72,dtype=int)
postprocessing.convertMatToTrack(noteMat.T,32,"original.mid")